# Parallelizing Reference Generation with Coiled
This notebook is a modified version of the `dask-nasa-nex.py` script that was used to generate the reference files. This example uses coiled to spin up a large # of small workers and reduces the computation time from 8+ hours on a 32 core 256Gb ram AWS instance to 30 minutes with 500 `t4g.small` workers.

In [ ]:
import dask.bag as db
import dask
import fsspec
import pandas as pd
from distributed import Client
from kerchunk.combine import MultiZarrToZarr
from kerchunk.hdf import SingleHdf5ToZarr

In [ ]:
cluster = coiled.Cluster(
    n_workers=500,
    worker_vm_types=["t4g.small"],
    spot_policy="spot_with_fallback",
    arm=True,
)
client = cluster.get_client()

In [ ]:
def _nasa_nex_df() -> pd.DataFrame:
    return pd.read_csv(
        "s3://carbonplan-share/nasa-nex-reference/nasa_nex_formatted.csv"
    )


def _GCM_scenarios(df: pd.DataFrame) -> pd.DataFrame:
    df = pd.read_csv("s3://carbonplan-share/nasa-nex-reference/nasa_nex_formatted.csv")
    colapsed_df = (
        df.groupby(["GCM", "scenario", "ensemble_member"])["variable"]
        .apply(list)
        .reset_index()
    )
    colapsed_df["variable"] = colapsed_df["variable"].apply(lambda x: list(set(x)))

    return colapsed_df


def read_catalog_file(catalog_url):
    return pd.read_csv(catalog_url)

In [ ]:
catalog_file_url = "s3://carbonplan-share/nasa-nex-reference/reference_catalog_prod.csv"
nasa_nex_df = _nasa_nex_df()
nasa_nex_catalog = _GCM_scenarios(nasa_nex_df)
nasa_nex_catalog["ID"] = nasa_nex_catalog["GCM"] + "_" + nasa_nex_catalog["scenario"]

# If we're going to write, we can use this to check what refs exist in our catalog
# kerchunk_ref_catalog = read_catalog_file(catalog_file_url)
# missing_refs_df = nasa_nex_catalog.merge(kerchunk_ref_catalog, how='outer', on='ID',indicator=True).query('_merge != "both"')[['GCM', 'scenario', 'ID']]

In [ ]:
# If we're not writing, we can use this for testing
missing_refs_df = nasa_nex_catalog

In [ ]:
missing_refs_df

In [ ]:
so = dict(mode="rb", anon=True, default_fill_cache=False, default_cache_type="first")


def build_reference_catalog(catalog_file_url: str):
    ref_list = [
        "s3://" + ref
        for ref in fs_read.ls(
            "s3://carbonplan-share/nasa-nex-reference/references_prod/"
        )
    ]
    ref_list.remove("s3://carbonplan-share/nasa-nex-reference/references_prod/")
    ref_df = pd.DataFrame({"ID": None, "url": ref_list})
    ref_df["ID"] = ref_df["url"].str.split("/", expand=True)[5]
    ref_df.to_csv(catalog_file_url, index=False)

In [ ]:
@dask.delayed
def mzz_test(refs):
    mzz = MultiZarrToZarr(
        refs,
        remote_protocol="s3",
        concat_dims=["time"],
        identical_dims=["lat", "lon"],
    ).translate()

    # We're skipping writing the parquet files for now, but
    # the entire reference is ~300 Mb, so shouldn't impact
    # Timing a bunch
    return mzz


@dask.delayed
def generate_json_reference(fil):
    fs_read = fsspec.filesystem("s3", anon=True, skip_instance_cache=True)

    with fs_read.open(fil, **so) as infile:
        h5chunks = SingleHdf5ToZarr(infile, fil, inline_threshold=300)
        return h5chunks.translate()  # outf


def gen_all_refs(row):
    GCM = row["GCM"]
    scenario = row["scenario"]

    file_pattern = nasa_nex_df.query(f"GCM == '{GCM}'  & scenario == '{scenario}'")

    refs = [generate_json_reference(fil) for fil in file_pattern["url"].to_list()]
    mzz = mzz_test(refs)
    return mzz

In [ ]:
row_list = [row for _, row in missing_refs_df.iterrows()]
tasks_alt = [(gen_all_refs)(row) for row in row_list]

In [ ]:
dask.compute(tasks_alt)

In [ ]:
client.shutdown()

## Update Catalog

In [ ]:
# build_reference_catalog(catalog_file_url)